In [10]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda,floatX=float32"
import theano
print(theano.config.device, theano.config.floatX)

import keras
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  
from sklearn.metrics import classification_report



cuda float32


In [11]:
class_names = ['1', '2', '3', '4', '5']
print(len(class_names))

5


In [12]:
K.set_learning_phase(1)

In [13]:
img_width, img_height = 71, 71
nb_train_samples = 26640
nb_validation_samples = 12630
epochs = 12
batch_size = 8
n_classes = 5

In [14]:
train_data_dir = "/datasets/DTLD/DTLD_crop/new_train/position"    # дальше там папки, а не сами фотографии!!!!
validation_data_dir = "/datasets/DTLD/DTLD_crop/new_test/position"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [15]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')


Found 40673 images belonging to 5 classes.
Found 9799 images belonging to 5 classes.


In [16]:
def build_model_Xception():
  base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = 5)
  
  for layer in base_model.layers:
    layer.trainable = True
      
  model = Model(inputs = base_model.input, outputs = base_model.output)
  return model

In [17]:
def classification_recall(y_true, y_pred):
    recall = list()
    for i in range(5):
        a = 0
        b = 0
        print(type(y_true))
        for j in range(y_true.size()[0]):
            if y_true[j] == i and y_pred == i:
                a += 1
            if y_true[j] == i and y_pred != i:
                b += 1
    recall.append([a/(a+b)])
    return recall

In [18]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

In [19]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-position-improvement-71-{epoch:02d}-{val_acc:.2f}.hdf5"
check = ModelCheckpoint(filepath, monitor = "val_acc", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr , check]

In [ ]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/12
3330/3330 [==============================] - 10564s 3s/step - loss: 0.4953 - acc: 0.8726 - mean_squared_error: 0.0390 - val_loss: 0.2040 - val_acc: 0.9449 - val_mean_squared_error: 0.0178
Epoch 2/12
3330/3330 [==============================] - 10216s 3s/step - loss: 0.2921 - acc: 0.9115 - mean_squared_error: 0.0258 - val_loss: 0.1673 - val_acc: 0.9563 - val_mean_squared_error: 0.0139
Epoch 3/12
3330/3330 [==============================] - 7808s 2s/step - loss: 0.2202 - acc: 0.9317 - mean_squared_error: 0.0200 - val_loss: 0.1402 - val_acc: 0.9567 - val_mean_squared_error: 0.0131
Epoch 4/12
3330/3330 [==============================] - 7288s 2s/step - loss: 0.1926 - acc: 0.9395 - mean_squared_error: 0.0177 - val_loss: 0.1164 - val_acc: 0.9635 - val_mean_squared_error: 0.0110
Epoch 5/12
3330/3330 [==============================] - 6957s 2s/step - loss: 0.1766 - acc: 0.9440 - mean_square

In [ ]:
model = load_model('weights-position-improvement-12-0.98.hdf5')

In [125]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [126]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [127]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse', precision, recall])

In [128]:
evaluation = model.evaluate_generator(validation_generator, verbose=1)

1225/1225 [==============================] - 1376s 1s/step - loss: 0.0488 - acc: 0.9846 - mean_squared_error: 0.0046 - precision: 0.9878 - recall: 0.9816


In [129]:
model.metrics_names

['loss', 'acc', 'mean_squared_error', 'precision', 'recall']

In [130]:
evaluation

[0.048828357186359646, 0.9845903, 0.0046078614, 0.9878134, 0.98163265]